# Test the Trend Index

## Check if index is up!

In [17]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [18]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['foodmap.isti.cnr.it'], http_auth=('elastic', 'changeme'), port=9200)
print es

<Elasticsearch([{u'host': u'foodmap.isti.cnr.it'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [7]:
# es.indices.delete(index='trend', ignore=[400, 404])

{u'acknowledged': True}

In [43]:
es.delete_by_query(index='trend', doc_type='tweet', body={
    'query': {
        "match_all" : {}
    }
})

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host=u'foodmap.isti.cnr.it', port=9200): Read timed out. (read timeout=10))

#### Count elements in index

In [44]:
es.count(index='trend', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 6571}

#### GET all docs from index

In [29]:
res = es.search(index="trend", doc_type='tweet', size=10, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits'][:20]:
    print r
    print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'840200729801416704', u'_source': {u'username': u'maslarovira', u'lang': u'und', u'text_categories': [u'hummus'], u'city': None, u'text': u'#hummus #bacall\xe0esqueixat #taronjasanguina #espinacs #yummy #brutal #espinelves #viladrau\u2026 https://t.co/bSnFsBlnyt', u'img_categories': None, u'hashtags': [u'#hummus', u'#bacall\xe0esqueixat', u'#taronjasanguina', u'#espinacs', u'#yummy', u'#brutal', u'#espinelves', u'#viladrau\u2026'], u'img_flag': True, u'timestamp_ms': u'1489154438816', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[2.392246, 41.849277], [2.392246, 41.896344], [2.454142, 41.896344], [2.454142, 41.849277]]]}, u'coords': [2.41838, 41.86547], u'id_str': u'840200729801416704', u'country': u'spain', u'id': 840200729801416704, u'media_url': None, u'created_at_day': 20170310, u'created_at_datetime': u'2017-03-10T14:00:38', u'created_at_month': 20170301}, u'_index': u'trend'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u

#### GET specific tweets

In [ ]:
es.get(index='trend', doc_type='tweet', id=880404048834416644)

In [42]:
results = es.search(index='trend', doc_type='tweet', size=100, body={
  'query': {
    'match': {
      'text_categories': 'hot_dog',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

print results
print len(results["hits"]["hits"])

{u'hits': {u'hits': [{u'_score': 4.6416197, u'_type': u'tweet', u'_id': u'849773409890447361', u'_source': {u'username': u'm90photo', u'lang': u'en', u'text_categories': [u'hot_dog'], u'city': None, u'text': u'I have an unnatural love for a #hotdog .\n@focalmark #liveauthentic #foodbeast #eeeeeats #eatfamous #feedfeed #dail\u2026\u2026 https://t.co/JxB2QIdi6V', u'img_categories': None, u'hashtags': [u'#hotdog', u'#liveauthentic', u'#foodbeast', u'#eeeeeats', u'#eatfamous', u'#feedfeed', u'#dail\u2026\u2026'], u'img_flag': True, u'timestamp_ms': u'1491436743601', u'bounding_box': None, u'coords': None, u'id_str': u'849773409890447361', u'country': u'united states', u'id': 849773409890447361, u'media_url': None, u'created_at_day': 20170405, u'created_at_datetime': u'2017-04-05T23:59:03', u'created_at_month': 20170401}, u'_index': u'trend'}, {u'_score': 4.6416197, u'_type': u'tweet', u'_id': u'849760863242375168', u'_source': {u'username': u'twintweet527', u'lang': u'en', u'text_categorie

#### TEST image classifier

In [ ]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data

### Index from file

In [ ]:
import json
import os
import sys
import time
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
from elasticsearch import Elasticsearch, RequestError
from processing.preprocess_tweet import process_tweet
from processing.twitter.Tweet import Tweet

In [ ]:
inputFile="/home/foodmap/data/food-tweets/food-tweets-2017-03-10.json.gz"
indexName="trend"

tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
numIndex = 0
for tweet in tweetsAsDict:
    i += 1
    if i % 10 == 0:
        print "Processed tweets: ", i
        print "Indexed tweets: ", numIndex

    new_tweet = process_tweet(tweet, forStream=False)
    if new_tweet is None:
        continue

    try:
        if i > 1480:
            print new_tweet
        # added request_timeout to avoid elasticsearch.exceptions.ConnectionTimeout
        es.index(index=indexName, doc_type='tweet', id=new_tweet["id"], body=new_tweet)
        numIndex += 1
        # print "Indexed tweet: ", new_tweet["id"]
    except RequestError as e:
        print "Couldn't index tweet id: ", new_tweet["id"]
        print e.status_code, e.message
        time.sleep(60)

print "Processed tweets: ", i
print "Indexed tweets: ", numIndex